## Preparação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Realização do carregamento do *dataset*;
- Aplicação dos filtros temporais (~ 5-50 Hz) - deixado como **tarefa**;
- Segmentação de dados nos domínios do tempo e da frequência em janelas deslizantes com sobreposição.

In [1]:
# Carregamento dos datasets

import numpy as np

data1 = np.load("dataset/Gabi.npy")
data2 = np.load("dataset/Jesse.npy")

data1.shape, data2.shape

((28, 1000, 4), (28, 1000, 4))

In [2]:
# Correção do número de eletrodos e unificação

# os dois primeiros eletrodos são funcionais
data1 = data1[:,:,:2]
data2 = data2[:,:,:2]

data = np.array([data1, data2])

# colocando os dados da dimensão final
data = data.transpose(0, 1, 3, 2)

data.shape

(2, 28, 2, 1000)

In [3]:
# funcões para aplicação dos filtros temporais

from scipy import signal

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

*Tarefa 1*: Realizem a aplicação dos filtros para toda a base.

In [4]:
# aplicação dos filtros

# codigo ...

In [5]:
# juntando participantes (apenas para demonstração)

data = data.reshape(data.shape[0] * data.shape[1], data.shape[2], data.shape[3])
data.shape

(56, 2, 1000)

#### Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). 

Os trechos de código a seguir dividem o dadom em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [6]:
from scipy.signal import stft

step = 29
segment = 64
print('Shape inicial:', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# # Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+participantes, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

Shape inicial: (56, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (56, 2, 33, 64) - (classes+participantes, canais, janelas, linhas)
Dominio da frequência:  (56, 2, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Tarefa 2*: separar os participantes e armazenar os dados em disco para a execução das próximas tarefas.

In [8]:
# Salvamento simples dos dados como está para uso no notebook "validation.ipynb" (somente professor).

np.save("dataset/segmented_data_t.npy", chunks_time)
np.save("dataset/segmented_data_f.npy", chunks_freq)